In [1]:
import os
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

In [4]:
def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

In [5]:
X_train = pd.read_csv('X_train_undersample.csv', index_col = ['Unnamed: 0'])
y_train = pd.read_csv('y_train_undersample.csv', index_col = ['Unnamed: 0'])
X_test = pd.read_csv('X_test_undersample.csv', index_col = ['Unnamed: 0'])
y_test = pd.read_csv('y_test_undersample.csv', index_col = ['Unnamed: 0'])

In [6]:
cat_col_names = ['Килип1', 'Килип4', 'Тромболит3', 'ГБ6', 'АКШ', 'Килип3', 'Килип2', 'Кл.вар.ИМ1', 'Хр.оч.инф2']
num_col_names = ['Creatin', 'ФУ1', 'ИндНор1', 'Возраст', 'Pсрлао(Адср)1', 'ПИ', 'Рла1']

In [7]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig

In [14]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform = "quantile_normal"
)

trainer_config = TrainerConfig(
    auto_lr_find = False,
    batch_size = 16,
    max_epochs = 10,
    gpus = 0,
    gradient_clip_val = 10,
    load_best = True
)

model_config = NodeConfig(
    task = "classification",
    num_layers = 2,
    num_trees = 1024,
    learning_rate = 0.1,
    embed_categorical = False,
    metrics = ["accuracy"],
)

tabular_model = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = OptimizerConfig(),
    trainer_config = trainer_config,
)

In [15]:
X_train['target'] = y_train
X_test['target'] = y_test

In [16]:
tabular_model.fit(train = X_train, test = X_test)

C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_data.py:2367: UserWarning: n_quantiles (1000) is greater than the total number of samples (140). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\pytorch_tabular\models\node\odst.py:152: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\uti

In [17]:
pred_df = tabular_model.predict(X_test)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
66,1.0,0.621190,1.695150,0.0,0.0,0.253860,0.0,0.0,1.448225,0.0,-0.232216,0.0,0.0,1.0,-0.347211,0.228140,0,0.238171,0.761829,1
490,0.0,0.671115,0.279454,0.0,0.0,-1.672989,0.0,0.0,1.371050,1.0,1.201122,0.0,1.0,0.0,0.038337,-0.282568,1,0.247625,0.752375,1
493,0.0,1.536485,1.412011,0.0,0.0,-1.458895,1.0,0.0,1.834102,1.0,-0.815108,0.0,0.0,0.0,0.963654,-0.282568,1,0.175707,0.824293,1
890,0.0,-0.796131,1.165575,1.0,0.0,-0.830675,0.0,1.0,-0.867036,1.0,0.621597,0.0,1.0,0.0,-0.733607,-0.282568,1,0.311030,0.688969,1
611,0.0,0.332733,-0.165048,0.0,0.0,0.117626,0.0,0.0,-1.638789,0.0,-0.502773,1.0,0.0,1.0,-1.041199,0.040012,0,0.796515,0.203485,0


In [18]:
print_metrics(X_test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.8636363636363636 | Holdout F1: 0.888888888888889


In [19]:
X_val = pd.read_csv('X_val.csv', index_col = 'Unnamed: 0')
y_val = pd.read_csv('y_val.csv', index_col = 'Unnamed: 0')

In [20]:
X_val['target'] = y_val

In [21]:
pred_df = tabular_model.predict(X_val)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
877,0.0,-1.264873,1.165575,1.0,1.0,-0.830675,0.0,1.0,-0.635509,0.0,0.621597,0.0,0.0,0.0,-0.733607,-0.282568,1,0.365595,0.634405,1
873,0.0,2.645934,-0.286824,0.0,1.0,-0.816612,1.0,0.0,1.448225,0.0,3.241676,0.0,0.0,0.0,-2.429174,-1.636425,1,0.298937,0.701063,1
247,0.0,2.690312,1.165575,0.0,1.0,-1.565942,0.0,0.0,1.525401,0.0,-0.940781,0.0,0.0,0.0,-2.429174,-0.282568,1,0.173569,0.826431,1
850,0.0,0.343828,-0.286824,0.0,0.0,-0.816612,0.0,0.0,-0.867036,1.0,-0.699190,0.0,1.0,0.0,0.963654,-0.600556,0,0.823948,0.176052,0
210,0.0,0.471414,-0.165048,0.0,0.0,-0.923659,0.0,0.0,0.136244,0.0,-0.492234,1.0,0.0,0.0,0.809435,0.435314,0,0.741067,0.258933,0


In [22]:
print_metrics(X_val['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.85 | Holdout F1: 0.8571428571428572


In [24]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform = "quantile_normal"
)

trainer_config = TrainerConfig(
    auto_lr_find = False,
    batch_size = 16,
    max_epochs = 10,
    gpus = 0,
    gradient_clip_val = 10,
    load_best = True
)

model_config = NodeConfig(
    task = "classification",
    num_layers = 3,
    num_trees = 1024,
    learning_rate = 0.1,
    embed_categorical = False,
    metrics = ["accuracy"],
)

tabular_model2 = TabularModel(
    data_config = data_config,
    model_config = model_config,
    optimizer_config = OptimizerConfig(),
    trainer_config = trainer_config,
)

In [25]:
tabular_model2.fit(train = X_train, test = X_test)

C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_data.py:2367: UserWarning: n_quantiles (1000) is greater than the total number of samples (140). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\olyav\AppData\Roaming\Python\Python38\site-packages\pytorch_tabular\models\node\odst.py:152: UserWarning: Data-aware initialization is performed on less than 1000 data points. This may cause instability.To avoid potential problems, run this model on a data batch with at least 1000 data samples.You can do so manually before training. Use with torch.no_grad() for memory efficiency.
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\pytorch_lightning\uti

In [26]:
pred_df = tabular_model2.predict(X_test)
pred_df.head()

,Килип1,ИндНор1,Creatin,Тромболит3,Килип4,ФУ1,ГБ6,АКШ,Возраст,Килип3,Рла1,Килип2,Кл.вар.ИМ1,Хр.оч.инф2,Pсрлао(Адср)1,ПИ,target,0_probability,1_probability,prediction
66,1.0,0.621190,1.695150,0.0,0.0,0.253860,0.0,0.0,1.448225,0.0,-0.232216,0.0,0.0,1.0,-0.347211,0.228140,0,0.356844,0.643156,1
490,0.0,0.671115,0.279454,0.0,0.0,-1.672989,0.0,0.0,1.371050,1.0,1.201122,0.0,1.0,0.0,0.038337,-0.282568,1,0.358984,0.641016,1
493,0.0,1.536485,1.412011,0.0,0.0,-1.458895,1.0,0.0,1.834102,1.0,-0.815108,0.0,0.0,0.0,0.963654,-0.282568,1,0.252499,0.747501,1
890,0.0,-0.796131,1.165575,1.0,0.0,-0.830675,0.0,1.0,-0.867036,1.0,0.621597,0.0,1.0,0.0,-0.733607,-0.282568,1,0.418081,0.581919,1
611,0.0,0.332733,-0.165048,0.0,0.0,0.117626,0.0,0.0,-1.638789,0.0,-0.502773,1.0,0.0,1.0,-1.041199,0.040012,0,0.792943,0.207057,0


In [27]:
print_metrics(X_test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.9090909090909091 | Holdout F1: 0.9230769230769231


In [34]:
X_val = pd.read_csv('X_val.csv', index_col = 'Unnamed: 0')
y_val = pd.read_csv('y_val.csv', index_col = 'Unnamed: 0')

In [35]:
X_val['target'] = y_val

In [36]:
pred_df = tabular_model2.predict(X_val)

In [37]:
print_metrics(X_val['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.8947368421052632 | Holdout F1: 0.9
